In [1]:
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import scipy.sparse as sp 

#After introducing new citation, does that new citation possibly link to another citation in the network?
## edges/citations in graph are positive [connection is there betw. papers/nodes]
## papers/nodes with no citations are negative [no citations, thus we predict]
## make 2 training and test sets [both of which apply for positive and negative]
## use AUC (Area Under Curve)

dataset = [0]
g = dataset[0]

Using backend: pytorch[21:30:14] /tmp/dgl_src/src/runtime/tensordispatch.cc:43: TensorDispatcher: dlopen failed: dlopen(/Users/devpatelio/miniconda3/envs/machine_learning/lib/python3.8/site-packages/dgl/tensoradapter/pytorch/libtensoradapter_pytorch_1.9.1.dylib, 0x0001): tried: '/Users/devpatelio/miniconda3/envs/machine_learning/lib/python3.8/site-packages/dgl/tensoradapter/pytorch/libtensoradapter_pytorch_1.9.1.dylib' (no such file), '/usr/local/lib/libtensoradapter_pytorch_1.9.1.dylib' (no such file), '/usr/lib/libtensoradapter_pytorch_1.9.1.dylib' (no such file)



In [3]:
u, v = g.edges() #u is source node, v is destination node
eids = np.arange(g.number_of_edges()) #make array from 0 -> num_edges in graph
eids = np.random.permutation(eids) #random shuffle

print(len(u), len(v))
print(eids)

test_size = int(len(eids)*0.1)
train_size = len(eids) - test_size
assert train_size+test_size == g.number_of_edges() #verify sizes

test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]] #take first 10% of corresp. source and destination nodes [pair of nodes = edges]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy()))) #create identity matrix with lengh of u source nodes, adjacency added on top with data u,v
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes()) 
    #assume matrix of all 1s
    #subtract adjacency matrix of graph connections [all 0s indicate presence of connection to another node]
    #subtract identity to include undirected/self-directed node connections
    #where ij != 0
